<div class="alert alert-block alert-info"; style = "color:black">
<h1>Monte Carlo Tree Search primenjen za kreaciju programa koji igra šah</h1>
<h2>> Nikola Matijević IN02-2018</h2>
<h2>> Marko Todorčević IN03-2018</h2>
</div>


<div class="alert alert-block alert-info" style = "color:black">
    <h1>Biblioteke koje smo koristili</h1>
    <h2>python-chess</h2> 
<p>je biblioteka za šah pisana za python, <br>sa generisanjem poteza, validacijom poteza, podrškom za različite formate šahovske table</p>

<h3>metode iz biblioteke python-chess</h3>

>chess.board.is_checkmate() <br> 
chess.board.is_insufficient_material() <br>
chess.board()<br>
chess.board.turn<br>
chess.board.generate_legal_moves()<br>
chess.copy.copy(chess.board)



## numpy & time
### metode

>np.log() <br>
np.random.random_sample()<br>
time.time() <br>
</div>

In [ ]:
import chess
import numpy as np
import time 

<h1 class="alert alert-block alert-info" style = "color:black"> MCTS node </h1>

In [ ]:
class MCTS_node:

    def __init__(self, board, parent):
        self.board = board
        self.parent = parent
        self.children = []

        self.wins = 0
        self.visits = 0

In [ ]:
    def selection(self):        
        ret_child = self.children[0]
        for child in self.children:
            childUCT = self.UCT(node = child)
            retchildUCT = self.UCT(node = ret_child)
            if(self.board.turn):
                if(childUCT < retchildUCT):
                    ret_child = child
            else:
                if(childUCT > retchildUCT):
                    ret_child = child
        return ret_child


In [ ]:
    @staticmethod
    def UCT(node):
        if node.visits == 0:
            if(node.board.turn):
                return 1
            else:
                return -1
        if(node.board.turn):#ako je true igra beli
            return node.wins / node.visits + (1 * np.log(node.parent.visits) / node.visits) ** (1 / 2)
        return (node.wins / node.visits - (1 * np.log(node.parent.visits) / node.visits) ** (1 / 2))


<div class="alert alert-block alert-success" style = "font-size:large; color:black">
Defult povratne vrednosti za neistražene čvorove: Inf/-Inf -> 0 -> 1/-1 -> 0.5/-0.5 -> 0.5/-1 -> 1/-1
<br>Faktor C u UCT: 2, 0.5, 0.01, 1
<br>UCT se sastoji iz eksplatacije i istraživanja</div>

In [ ]:
    def expansion(self):
        legal_moves = list(self.board.generate_legal_moves())
        
        for move in legal_moves:
            # a new board must be made for every node, and must then be updated with the current move
            new_board = chess.copy.copy(self.board)
            new_board.push(move)
            self.children.append(MCTS_node(new_board, self))


In [ ]:
    def simulate_batch(self):
        # number of playouts, fit for change
        no_playouts = 1
        
        for _ in range(no_playouts):
            # do a simulation from the current board state, sum the returned value to the 
            a = self.simulation()
            # a is so far, either 0 or 1
            self.wins += a
            # visits gets incremented regardless of wheter or not the simulation resulted in a win
            self.visits += 1


<div class="alert alert-block alert-success" style = "font-size:x-large; color:black">
<b>no_playouts: 25 -> 5 -> 1</b></div>

In [ ]:
    def simulation(node):
        new_tmp_board = chess.copy.copy(node.board)
        n = 20;
        while(n):
            n -= 1
            if(new_tmp_board.is_check()):
                break
            legal = list(new_tmp_board.generate_legal_moves())
            if(len(legal) == 0):
                break
            new_tmp_board.push((legal[(int)(np.random.random_sample()*(len(legal)-1))]))
        
        if(n == 0):
            return 0
        if(new_tmp_board.turn):
            return 1
        if(~new_tmp_board.turn):
            return -1
        return 0


![relation](relation.png)

In [ ]:
    def back_propagation(self):
        # used to iterate over parents
        tmp = self.parent
        
        while(tmp != None):
            tmp.visits += self.visits
            tmp.wins += self.wins
            tmp = tmp.parent


In [ ]:
import chess
import numpy as np
import time 
    
class MCTS_node:
    
    def __init__(self, board, parent):
        self.board = board
        self.parent = parent
        self.children = []
        
        self.wins = 0
        self.visits = 0
        
    @staticmethod
    def UCT(node):
        if node.visits == 0:
            if(node.board.turn):
                return 1
            else:
                return -1
        if(node.board.turn):#ako je true igra beli
            return node.wins / node.visits + (1 * np.log(node.parent.visits) / node.visits) ** (1 / 2)
        return (node.wins / node.visits - (1 * np.log(node.parent.visits) / node.visits) ** (1 / 2))
        
    # returns the child whose UCT value is the highest of all the other children
    # if there are multiple child nodes with the same value then the node that was found first will be returned
    # maybe fix this and return one random from all the ones with the max value
    def selection(self):        
        ret_child = self.children[0]
        for child in self.children:
            childUCT = self.UCT(node = child)
            retchildUCT = self.UCT(node = ret_child)
            if(self.board.turn):
                if(childUCT < retchildUCT):
                    ret_child = child
            else:
                if(childUCT > retchildUCT):
                    ret_child = child
        return ret_child
    
    
    def simulation(node):
        new_tmp_board = chess.copy.copy(node.board)
        n = 20
        while(n):
            n -= 1
            if(new_tmp_board.is_check()):
                break
            legal = list(new_tmp_board.generate_legal_moves())
            if(len(legal) == 0):
                break
            new_tmp_board.push((legal[(int)(np.random.random_sample()*(len(legal)-1))]))
        
        if(n == 0):
            return 0
        if(new_tmp_board.turn):
            return -1
        if(~new_tmp_board.turn):
            return 1
        return 0
       
    # expansion is done by generating all legal moves and then appending them to the tree
    def expansion(self):
        legal_moves = list(self.board.generate_legal_moves())
        
        for move in legal_moves:
            # a new board must be made for every node, and must then be updated with the current move
            new_board = chess.copy.copy(self.board)
            new_board.push(move)
            self.children.append(MCTS_node(new_board, self))
    
    
    # updates all the node that are on the path from the current node, up until it reaches the root node
    def back_propagation(self):
        # used to iterate over parents
        tmp = self.parent
        
        while(tmp != None):
            tmp.visits += self.visits
            tmp.wins += self.wins
            tmp = tmp.parent
            
    def simulate_batch(self):
        # number of playouts, fit for change
        no_playouts = 1
        
        for _ in range(no_playouts):
            # do a simulation from the current board state, sum the returned value to the 
            result = self.simulation()
            # a is so far, either 0 or 1
            self.wins += result
            # visits gets incremented regardless of wheter or not the simulation resulted in a win
            self.visits += 1

<div class="alert alert-block alert-success" style = "font-size:x-large; color:black">
<b>Sama igra:</b> vs stockfish -> vs self</div>

In [ ]:
def play(num_moves = 20, time_for_move = 3):
    root = MCTS_node(chess.Board(), None)
    while ((num_moves) and ~(root.board.is_checkmate())):
        num_moves -= 1        
        root.parent = None
        tmp = root
        if(root.board.is_checkmate()):
            return root.board
        start_time = time.time()
        while (time.time() - start_time < time_for_move):
            tmp = root        
            while (len(tmp.children) != 0):
                tmp = tmp.selection()
                
            tmp.expansion()
            if(tmp.visits == 0):
                tmp.simulate_batch()
                tmp.back_propagation()

        #for node in root.children:
        #    print("{} {} {}".format(node.visits, node.wins, node.UCT(node)))
        root = root.selection()
        if(root.board.is_checkmate() or root.board.is_insufficient_material()):
            return root.board
        print(root.board);
        #print("\n")
        print(num_moves)
    return root.board   

            
play(100,1)

### parametri 50 poteza 6 sekund razmišljanja
![parametri_50_poteza_6_sekundi_3](parametri_50_poteza_6_sekundi_3.png)
![parametri_50_poteza_6_sekundi_2](parametri_50_poteza_6_sekundi_2.png)
![parametri_50_poteza_6_sekundi](parametri_50_poteza_6_sekundi.png)

### pobede belog 1000 poteza 1 sekund razmišljanja


![white_win_1000_moves_1_s](white_win_1000_moves_1_s.png)
![white_win_1000_moves_1_s_2](white_win_1000_moves_1_s_2.png)
![white_win_1000_moves_1_s_3](white_win_1000_moves_1_s_3.png)